In [1]:
import cv2
import numpy as np
import os
import pandas as pd
import pydicom

from pydicom.pixel_data_handlers.util import apply_voi_lut
from tqdm.notebook import tqdm as tqdm
import warnings
warnings.filterwarnings('ignore')

class Convert:
        def __init__(self,ip_path:str):
            self.ip_path=ip_path
            
        def read_img(self,patient_id:str,img_id: str,voi_lut: bool=False,fix_monochrome: bool=True):
            #reading the image
            dicom = pydicom.read_file(os.path.join(self.ip_path,patient_id,img_id+".dcm"))

            # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to "human-friendly" view
            if voi_lut:
                data = apply_voi_lut(dicom.pixel_array, dicom)
            else:
                data = dicom.pixel_array

            # depending on this value, X-ray may look inverted - fix that:
            # i.e. to ensure consistent color for cancer and rest of the image
            if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
                data = np.amax(data) - data

            data = data - np.min(data)
            data = data / np.max(data)
            
            data = (data * 255).astype(np.uint8)

            return data

        def resize_img(self,img: np.array,image_size: int):
            height, width = img.shape
            scale = image_size / min(width, height)
            w = int(width * scale + 0.5)
            h = int(height * scale + 0.5)

            if scale > 1.0:
                interpolation=cv2.INTER_CUBIC
            else:
                interpolation=cv2.INTER_AREA

            return cv2.resize(img, (w, h), interpolation=interpolation)
    
if __name__ == "__main__":
    op_folder="D:/temp/test_new/"
    convert=Convert("D://Project_data//test_images/")
    df=pd.read_csv('D:/Project_data/test.csv') 
    total_rows=range(len(df))
    size=512
    
    if not os.path.exists(op_folder):
        os.mkdir(op_folder)
    
    for row in tqdm(total_rows, dynamic_ncols=True):
        img=convert.read_img(str(df['patient_id'][row]),str(df['image_id'][row]))

        cv2.imwrite(op_folder+str(df['patient_id'][row])+'_'+str(df['image_id'][row])+".png", 
                    convert.resize_img(img,size))
    

  0%|                                                                                            | 0/4 [00:00<…